In [1]:
import os
import re
import itertools
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks
from tabulate import tabulate

### Building the dataframe with the results

In [2]:
datapath = '../data/img_labeling_3rd_round/'

#### Reading orginal files

In [3]:
label_files = sorted(os.listdir(datapath))
label_files = [f for f in label_files if 'csv' in f and "_" in f]
print(f"{len(label_files)} .csv files in the folder\n")
#for f in label_files:
#    print(f)

47 .csv files in the folder



#### Extracting subjects

In [4]:
subjects = {x[:x.find("_")] for x in label_files}
print(f"{len(subjects)} diferent subjects")

20 diferent subjects


#### Consolidating files per user

In [5]:
n_files = []
for subject in subjects:
    s_files = [os.path.join(datapath, f) for f in label_files if f.startswith(subject)]
    n_file = os.path.join(datapath, subject + ".csv")
    n_files.append(n_file)
    with open(n_file, "w") as fw:
        for s in s_files:
            with open(s, "r") as fr:
                lines = fr.read()
                lines = lines.replace('"','')
                lines = lines.replace("[Appealing,Non-appealing]", "[Non-appealing]")   ## Choosing the last chosen class for ambiguous
                lines = lines.replace("[Non-appealing,Appealing]", "[Appealing]")
                fw.write(lines+"\n")

#### Reading CSV files into Dataframes

In [6]:
list_dfs = []
for idx, n_file in enumerate(n_files):
    user = re.findall("|".join(subjects), n_file)[0]
    userid = idx + 1
    print(f"{userid}: \t {user}")
    df = pd.read_csv(n_file, names=['image_name', 'class','w','h'])
    df['user'] = user
    df['userid'] = userid
    df.drop(['w','h'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    list_dfs.append(df)

1: 	 245DC3767EBDBF721824F2F09B4251D5E0063494
2: 	 B59DE8F9A0D4C9AADABAFF7EA6A4F61BF89861DA
3: 	 38205FF03AFF19E008FA123615336213A0ED33AB
4: 	 F96F6ACF9B40EDA60D4D48FBED7B3512E1C8E48D
5: 	 BE887E16DEB41D06FACD481B5BFE3AF6FDF6F8CB
6: 	 43166EAC7DE5C6D48FF646B1B12BEB46AC7CA375
7: 	 5FFB96918AA02D6E1EF82257A2828F9A4FD9C702
8: 	 F0E0055FB33661B91E7C55DAFB0246BCB43D7D8D
9: 	 8D396D662D61B164860C081535E30C3E32934406
10: 	 5FEF4C8B719CABDF32C46A4C9C9316376DF7B512
11: 	 C003DF0C2EC67886EA6BC835682E3182A2126135
12: 	 867D8B4F797167AA1CA27E00F76B51C63A7278ED
13: 	 73396439E5B11BBCA57A8FCB53FBB6F0FCECE794
14: 	 C91C4E40D11000EB5E5BE0EDBE12C74A54FAA40B
15: 	 3AB6D9AAC2244ECC4F595743DF6EB54686DE31A0
16: 	 A6241D7982404859C8D816D8C4A61DC506145A50
17: 	 C733B58CB8AE244885A97E86B09D896A5633674C
18: 	 F9CC3D73B8F36057257097AFFE29B884000BABB5
19: 	 6438BC09053F6B5215CA1FC743AB0E3CF2B53984
20: 	 C6C77B9A8025D969F0E14BEBBEECB8B82BAAF284


#### Concatenating Dataframes

In [7]:
df_labeling = pd.concat(list_dfs)
df_labeling['id_image'] = df_labeling['image_name'].apply(lambda x:x[:-4])
df_labeling = df_labeling[['userid','user', 'image_name', 'id_image', 'class']]
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18156 entries, 0 to 1010
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userid      18156 non-null  int64 
 1   user        18156 non-null  object
 2   image_name  18156 non-null  object
 3   id_image    18156 non-null  object
 4   class       18156 non-null  object
dtypes: int64(1), object(4)
memory usage: 851.1+ KB


#### Fixing classes names

In [8]:
print(df_labeling['class'].unique())
df_labeling["class"] = df_labeling["class"].str.strip("[] ")
df_labeling["class"].replace({'Npn-appealing':"Non-appealing"}, inplace=True)
df_labeling['class'].value_counts()

['[Non-appealing]' '[Appealing]' '[Appealing]    ' '[Npn-appealing]']


Non-appealing    11076
Appealing         7080
Name: class, dtype: int64

In [9]:
df_labeling.head()

,userid,user,image_name,id_image,class
0,1,245DC3767EBDBF721824F2F09B4251D5E0063494,1.jpg,1,Non-appealing
1,1,245DC3767EBDBF721824F2F09B4251D5E0063494,2.jpg,2,Appealing
2,1,245DC3767EBDBF721824F2F09B4251D5E0063494,3.jpg,3,Non-appealing
3,1,245DC3767EBDBF721824F2F09B4251D5E0063494,4.jpg,4,Non-appealing
4,1,245DC3767EBDBF721824F2F09B4251D5E0063494,5.jpg,5,Non-appealing


#### Checking images classified

In [10]:
df_labeling['image_appearances'] = df_labeling.groupby('id_image')['id_image'].transform('count')

In [11]:
print(f"{len(df_labeling['image_name'])} images were classified")
print(f"{len(df_labeling['image_name'].unique())} unique images were classified\n")
num_images = 0
image_appearances = df_labeling.image_appearances.value_counts().sort_index(ascending=True)
for idx in image_appearances.index:
    print(f'{image_appearances[idx]} images were classified {idx} times')
    num_images += image_appearances[idx]

18156 images were classified
1012 unique images were classified

11 images were classified 11 times
165 images were classified 15 times
8016 images were classified 16 times
18 images were classified 18 times
266 images were classified 19 times
9680 images were classified 20 times


#### Keeping the images that were classified by all subjects (optional)

In [12]:
#df_labeling = df_labeling[df_labeling['image_appearances'] == 20]
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18156 entries, 0 to 1010
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   userid             18156 non-null  int64 
 1   user               18156 non-null  object
 2   image_name         18156 non-null  object
 3   id_image           18156 non-null  object
 4   class              18156 non-null  object
 5   image_appearances  18156 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 992.9+ KB


### Analysing the [inter-annotator agreement](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) on the results  

### $\kappa = (p_o - p_e) / (1 - p_e)$

#### where $p_o$ is the empirical probability of agreement on the label assigned to any sample (the observed agreement ratio), and $p_e$ is the expected agreement when both annotators assign labels randomly. $p_e$ is estimated using a per-annotator empirical prior over the class labels

In [13]:
l1 = df_labeling.userid.unique()
iter_users = itertools.product(l1,l1)
df_iaa = pd.DataFrame(index=l1, columns=l1)

In [14]:
for user1,user2 in iter_users:
    classesA = df_labeling.loc[(df_labeling.userid == user1),['id_image', 'class']]
    classesA.sort_values(by=['id_image'], inplace=True)

    classesB = df_labeling.loc[(df_labeling.userid == user2),['id_image', 'class']]
    classesB.sort_values(by=['id_image'], inplace=True)

    classesAB = pd.merge(classesA, classesB, on=['id_image'])
    #classesAB.drop_duplicates(subset='id_image', keep = 'first', inplace=True) 
    classesAB.drop('id_image', axis=1, inplace=True)
    classesAB.dropna(inplace=True)

    agreement = cks(classesAB['class_x'], classesAB['class_y'])
    df_iaa.loc[user1,user2] = f'{agreement:.3f}/({len(classesAB)})'

#print(tabulate(df_iaa, headers='keys', tablefmt='psql'))

In [15]:
df_iaa.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,1.000/(1012),-0.011/(1012),0.022/(1010),0.003/(1012),-0.009/(1012),-0.004/(1008),-0.061/(1002),0.081/(495),-0.054/(1011),-0.024/(500),0.039/(1009),0.030/(499),-0.017/(1007),0.022/(500),0.023/(1012),-0.017/(1011),0.000/(1010),0.021/(1011),0.011/(1012),0.020/(1011)
2,-0.011/(1012),1.000/(1012),-0.310/(1010),0.478/(1012),-0.019/(1012),0.457/(1008),0.121/(1002),-0.082/(495),0.114/(1011),0.044/(500),-0.130/(1009),0.428/(499),0.260/(1007),0.084/(500),0.343/(1012),0.182/(1011),0.058/(1010),-0.215/(1011),0.606/(1012),0.174/(1011)
3,0.022/(1010),-0.310/(1010),1.000/(1010),-0.304/(1010),0.019/(1010),-0.248/(1006),0.082/(1000),0.295/(493),-0.124/(1009),0.058/(498),0.277/(1007),-0.236/(497),-0.199/(1005),-0.011/(498),-0.123/(1010),-0.061/(1009),0.022/(1008),0.345/(1009),-0.401/(1010),-0.004/(1009)
4,0.003/(1012),0.478/(1012),-0.304/(1010),1.000/(1012),-0.046/(1012),0.385/(1008),0.122/(1002),-0.099/(495),0.135/(1011),0.064/(500),-0.071/(1009),0.502/(499),0.269/(1007),0.126/(500),0.277/(1012),0.201/(1011),0.065/(1010),-0.169/(1011),0.520/(1012),0.273/(1011)
5,-0.009/(1012),-0.019/(1012),0.019/(1010),-0.046/(1012),1.000/(1012),-0.052/(1008),-0.015/(1002),0.052/(495),0.016/(1011),0.036/(500),0.034/(1009),-0.007/(499),0.020/(1007),0.043/(500),-0.036/(1012),0.007/(1011),-0.017/(1010),0.017/(1011),-0.042/(1012),0.034/(1011)


In [ ]:
df_iaa.to_excel("../data/outputs/round3_results.xls")

In [16]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18156 entries, 0 to 1010
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   userid             18156 non-null  int64 
 1   user               18156 non-null  object
 2   image_name         18156 non-null  object
 3   id_image           18156 non-null  object
 4   class              18156 non-null  object
 5   image_appearances  18156 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 992.9+ KB


In [17]:
df_labeling['round'] = 3
df_labeling.drop('image_appearances', inplace=True, axis=1)

In [18]:
df_labeling.to_hdf('../data/df_labeling.hdf', key='round3')